In [14]:
import numpy as np
from scipy.linalg import block_diag
import itertools
import scipy as sc
import scipy.sparse as scs
from joblib import Parallel, delayed
import sys
import copy

In [15]:
S0 = np.array([0,0,0])
S1 = np.array([1,1,0])
S2 = np.array([1,0,1])
S3 = np.array([0,1,1])

R = np.array([S0,S1,S2,S3])

In [16]:
def boolean_combinations(n):
    return [
        *itertools.product(
            *[[0, 1, 2, 3] for _ in range(n)]
    )]

In [17]:
def ten2four(n):
    n = n % (4294967296)
    s = np.base_repr(n,4)
    return np.array(list(s.zfill(16)),dtype=int)

In [18]:
def get_plaquete_h():
    W1 = 0
    W2 = 0
    W3_1 = 0
    W3_2 = 0
    W4 = 0
    W5 = 0
    W6 = 0
    W7 = 0
    W8 = 0

    T1 = -128
    T2 = 32
    T3 = 32
    T4 = 32
    T5 = -32
    T6 = -32
    T7 = -32
    T8 = 32

    H1 = np.matrix([[W1, T1],[T1, W1]])
    H2 = np.matrix([[W2, T2],[T2, W2]])
    H3 = np.matrix([[W3_1, T3],[T3.conjugate(), W3_2]])
    H4 = np.matrix([[W4, T4],[T4, W4]])
    H5 = np.matrix([[W5, T5],[T5, W5]])
    H6 = np.matrix([[W6, T6],[T6, W6]])
    H7 = np.matrix([[W7, T7],[T7, W7]])
    H8 = np.matrix([[W8, T8],[T8, W8]])

    H1_s = np.kron(np.eye(1,dtype=int),H1)
    H2_s = np.kron(np.eye(6,dtype=int),H2)
    H3_s = np.kron(np.eye(6,dtype=int),H3)
    H4_s = np.kron(np.eye(3,dtype=int),H4)
    H5_s = np.kron(np.eye(6,dtype=int),H5)
    H6_s = np.kron(np.eye(6,dtype=int),H6)
    H7_s = np.kron(np.eye(3,dtype=int),H7)
    H8_s = np.kron(np.eye(1,dtype=int),H8)

    H_p = block_diag(H1_s, H2_s, H3_s, H4_s, H5_s, H6_s, H7_s, H8_s)

    basis_trafo = np.zeros((4,4,4,64))

    ones = [
        (3,2,1,0),
        (0,0,0,1),

        (1,0,0,2),
        (2,2,1,3),
        (3,3,1,4),
        (0,1,0,5),
        (0,3,0,6),
        (3,1,1,7),
        (3,2,2,8),
        (0,0,3,9),
        (0,0,2,10),
        (3,2,3,11),
        (1,2,1,12),
        (2,0,0,13),

        (1,1,0,14),
        (2,3,1,15),
        (3,0,1,16),
        (0,2,0,17),
        (0,3,3,18),
        (3,1,2,19),
        (3,2,0,20),
        (0,0,1,21),
        (2,0,2,22),
        (1,2,3,23),
        (0,2,1,24),
        (3,0,0,25),

        (1,2,0,26),
        (2,0,1,27),
        (3,0,2,28),
        (0,2,3,29),
        (0,3,1,30),
        (3,1,0,31),

        (1,3,0,32),
        (2,1,1,33),
        (3,3,2,34),
        (0,1,3,35),
        (0,3,2,36),
        (3,1,3,37),
        (1,2,2,38),
        (2,0,3,39),
        (1,0,2,40),
        (2,2,3,41),
        (1,3,1,42),
        (2,1,0,43),

        (1,3,3,44),
        (2,1,2,45),
        (3,3,0,46),
        (0,1,1,47),
        (2,3,2,48),
        (1,1,3,49),
        (0,2,2,50),
        (3,0,3,51),
        (1,1,2,52),
        (2,3,3,53),
        (1,0,1,54),
        (2,2,0,55),

        (1,0,3,56),
        (2,2,2,57),
        (3,3,3,58),
        (0,1,2,59),
        (2,3,0,60),
        (1,1,1,61),

        (1,3,2,62),
        (2,1,3,63),
    ]

    for i in ones:
        basis_trafo[i]=1


    prod1 = np.tensordot(basis_trafo,H_p,axes=(3,0))
    H_trafo_tensor = np.tensordot(prod1, basis_trafo, axes=(3,3)) # can be flattened as above

    return H_trafo_tensor

In [19]:
def get_plaquete_sparse():
    row = []
    column = []
    val = []

    h = get_plaquete_h()

    keys = boolean_combinations(3)

    for i in keys:
        for j in keys:
            a = h[i][j]
            if a != 0:
                val.append(a)
                row.append(i)
                column.append(j)
    return np.array(val), np.array(row), np.array(column)

In [20]:
gauge_indices = np.loadtxt("gauge_indices_red.txt").astype(int)
gauge_indices_base4 = np.array([ten2four(i) for i in gauge_indices])

In [21]:
def index_finder(index_4,i,j,k):
    cond = (gauge_indices_base4[:,i]==index_4[0])
    cond = (gauge_indices_base4[:,j]==index_4[1])*cond
    cond = (gauge_indices_base4[:,k]==index_4[2])*cond

    return np.where(cond==True)

In [22]:
def generate_space_h_mag(p):
    p_indices = np.array([[ 0, 5, 4],[ 1, 6, 5],[ 2, 7, 6],[ 4, 9, 8],[ 5,10, 9],[ 6,11,10],[ 8,13,12],[ 9,14,13],[10,15,14],[3,4,7],[7,8,11],[11,12,15]])
    vals = []
    rows = []
    cols = []

    p_val, p_row, p_column = get_plaquete_sparse()
    i,j,k = p_indices[p]
    for v in range(64):
        temp_row = index_finder(p_row[v],i,j,k)[0]
        temp_col = index_finder(p_column[v],i,j,k)[0]
        l = len(temp_row)
        if l>0:
            vals.append(np.ones(l)*p_val[v])
            rows.append(temp_row)
            cols.append(temp_col)
    
    return scs.coo_array((np.concatenate(vals), (np.concatenate(rows), np.concatenate(cols))), shape=(32768, 32768))

In [23]:
results = Parallel(n_jobs=12)(delayed(generate_space_h_mag)(i) for i in range(12))

In [24]:
h_mag = np.sum(results)

In [25]:
scs.save_npz("h_mag",h_mag)